Projet Maths-Info
===============

Myrtille DAVID et Bérénice SINOPOLI--PAL




In [23]:
import autograd
from autograd import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

Amorce
------

D'après le théorème des valeurs intermédiaires, si la fonction $f_{1}:x \mapsto f(x,0)$ est continue et que $f(0,0)\leq c\leq f(1,0)$ alors il existe $t\in [0,1]$ tel que $f(t,0)=c$.
Plusieurs algorithmes sont envisageables pour trouver ce t. Comme la méthode de Newton ne fonctionne pas avec toutes les fonctions, on a préféré ici une méthode par dichotomie.

In [18]:
def find_seed(g, c=0, eps=2**(-26)):
    def f(x,y):
        return g(x,y)-c
    a,b=0,1
    m=(a+b)/2
    
    if f(0,a)*f(0,b)<0 :
        while abs(f(0,m))>eps:
            if f(0,a)*f(0,m)<0:
                b = m
            elif f(0,b)*f(0,m)<0:
                a = m
            m = (a+b)/2
    
        return m
    elif f(0,a)==0:
        return a
    elif f(0,b)==0:
        return b
    else :
        return None

Propagation : première version
-------------

Voici une première ébauche de notre programme simple_contour. On utilise ici le fait que le gradient est en tout point orthogonal aux lignes de niveau. 

**Principe :** on définit une fonction grad_f qui calcule le gradient de la fonction en un point donné. On trace ensuite la ligne de niveau point par point : on calcule le gradient, on ajoute un point dans la direction orthogonale éloigné d'une distance $\delta$ du précédent.

**Le problème majeur de ce programme :** à chaque fois que l'on ajoute un point, l'erreur s'accumule et on peut s'éloigner beaucoup de la ligne de niveau initiale.

In [19]:
def simple_contour_brouillon(f, c=0.0, delta=0.01):
    
    def grad_f(x,y):
        g = autograd.grad
        return np.r_[g(f,0)(x,y), g(f,1)(x,y)]
        
    x=[]
    y=[]
    
    a = find_seed(f,c)
    if a is None :
        return x,y
    else:
        x.append(0.0)
        y.append(a)
        
    while len(x)<1000: #quelle condition pour arrêter de tracer la ligne de niveau ?
        gr = grad_f(x[-1],y[-1])
        d = [gr[1],-gr[0]]
        L=np.sqrt(d[0]**2+d[1]**2)
        d[0]=d[0]*delta/L
        d[1]=d[1]*delta/L
        
        x.append(x[-1]+d[0])
        y.append(y[-1]+d[1])
        print(f(x[-1],y[-1]))
        
    plt.plot(x,y)
    plt.show()
    return x,y
    

Propagation : version finale
------------------

Ce programme fonctionne selon le même principe que le précédent. Voici les ajustements proposés : 

**Correction de l'erreur :** à chaque fois que l'on calcule un nouveau point, on vérifie qu'il n'est pas trop éloigné de la ligne de niveau souhaitée en comparant $f(x,y)$ à $c$. Si $\left | f(x,y)-c) \right |> 10^{-4}$, alors on se "déplace" dans la direction donnée par le gradient (dans le sens du gradient ou le sens opposé selon le signe de la différence) pour se rapprocher de la ligne de niveau. A chaque itération, on vérifie que la différence $\left | f(x,y)-c) \right |> 10^{-4}$ n'augmente pas ; sinon, on sort de la boucle.

**Quand arrêter de tracer la ligne de niveau ?** 
Plusieurs cas de figure sont envisageables :
- soit la ligne de niveau rencontre un des bords du carré; on s'arrête.
- soit la ligne de niveau ne rencontre jamais de bord; dans ce cas, pour éviter de calculer indéfiniment de nouveaux points, on choisit arbitrairement de limiter le nombre de points de la ligne à 10000.

In [20]:
    
def simple_contour(f, c=0.0, delta=0.01):
    
    def grad_f(x,y): #on définit la fonction gradient
        g = autograd.grad
        return np.r_[g(f,0)(x,y), g(f,1)(x,y)]
        
    x=[]
    y=[]
    
    a = find_seed(f,c) #on cherche le premier point à l'aide du programme find_seed
    
    if a is None :
        return x,y
    else:
        x.append(0.0)
        y.append(a)
        
    while len(x)<10000: #on limite le nombre de points au cas où la ligne de niveau n'atteigne jamais de bord du carré
        gr = grad_f(x[-1],y[-1]) #on calcule le gradient du dernier point trouvé
        d = [gr[1],-gr[0]] #on calcule le vecteur orthogonal
        L=np.sqrt(d[0]**2+d[1]**2)
        d[0]=d[0]*delta/L #on change la norme de d pour qu'elle soit proche de delta
        d[1]=d[1]*delta/L
        
        x2=x[-1]+d[0] #on calcule le nouveau point
        y2=y[-1]+d[1]
        
        différence= f(x2,y2)-c #on vérifie que le nouveau point calculé ne s'éloigne pas trop de la ligne de niveau
        while abs(différence)>10**(-4):
            différence_ref=différence
            t=grad_f(x2,y2)
            norme_t=np.sqrt(t[0]**2+t[1]**2)
            
            if différence>0: #si f(x2,y2)>c alors on se déplace dans le sens opposé au gradient
                x3=x2-t[0]/norme_t*(delta/50)
                y3=y2-t[0]/norme_t*(delta/50)
            else: #sinon on se déplace dans le sens du gradient
                x3=x2+t[0]/norme_t*(delta/50)
                y3=y2+t[0]/norme_t*(delta/50)
            différence= f(x3,y3)-c
            
            if différence>différence_ref: #si on s'éloigne de la ligne de niveau, on arrête la correction
                break
                
            x2=x3
            y2=y3
                
        
        x.append(x2)
        y.append(y2)
        
        if (abs(x[-1]-1)<delta) or ((abs(x[-1])<delta) and len(x)>10) or (abs(y[-1]-1)<delta) or (abs(y[-1])<delta): #on s'arrête quand on atteint les bords
            break
        
    plt.plot(x,y)
    return(x,y)
    

Test de simple_contour
-------------

On teste simple_contour sur plusieurs fonctions définies ci-dessous.

In [21]:
def h2(x,y):
    return x+y**2-0.47

In [22]:
simple_contour(h2,0.25)
simple_contour(h2,0.35)
simple_contour(h2,0.45)

plt.show()

AttributeError: module 'matplotlib.colors' has no attribute 'to_rgba'